# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Para Aplicações de IA com PyTorch e Lightning</font>

## <font color='blue'>Mini-Projeto 2</font>
### <font color='blue'>Sistema de Recomendação de Aplicativos de Contabilidade com Graph Attention Network - GAT</font>

![DSA](imagens/MP2.png)

## Instalando e Carregando os Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
!pip install -q torch==2.0.0

https://pytorch-geometric.readthedocs.io/en/latest/

In [4]:
!pip install -q torch_geometric==2.3.1

In [5]:
# Imports
import pandas as pd
import numpy as np
import sklearn
import torch
import torch_geometric
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.utils import to_undirected
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

pandas         : 1.4.4
numpy          : 1.23.5
torch_geometric: 2.3.1
sklearn        : 1.0.2
torch          : 2.0.0



## Preparando o Conjunto de Dados

Aqui, você deve carregar seus próprios dados de usuários, aplicativos (ou qualquer outra coisa) e avaliações. Como exemplo, criaremos um dataset fictício.

In [7]:
# Dataframe de usuários
users = pd.DataFrame({"user_id": [0, 1, 2, 3]})

# Dataframe de apps
aplicativos = pd.DataFrame({"app_id": [0, 1, 2]})

# Dataframe de avaliações (ratings)
ratings = pd.DataFrame({"user_id": [0, 1, 1, 2, 3],
                        "app_id": [0, 0, 1, 2, 2],
                        "rating": [4, 5, 3, 2, 4]})

In [8]:
users

,user_id
0,0
1,1
2,2
3,3


In [9]:
aplicativos

,app_id
0,0
1,1
2,2


In [10]:
ratings

,user_id,app_id,rating
0,0,0,4
1,1,0,5
2,1,1,3
3,2,2,2
4,3,2,4


In [11]:
# Converta os IDs dos aplicativos para evitar confusão com os IDs dos usuários
ratings["app_id"] += users.shape[0] - 1

In [12]:
ratings

,user_id,app_id,rating
0,0,3,4
1,1,3,5
2,1,4,3
3,2,5,2
4,3,5,4


## Pré-Processamento dos Dados no Formato de Grafo

In [13]:
# Divide os dados em conjuntos de treinamento e teste
train_ratings, test_ratings = train_test_split(ratings, test_size = 0.2, random_state = 42)

In [14]:
# Prepara os nodes de treino
train_source_nodes = torch.tensor(train_ratings["user_id"].values, dtype = torch.long)
train_target_nodes = torch.tensor(train_ratings["app_id"].values, dtype = torch.long)

In [15]:
train_source_nodes

tensor([3, 1, 0, 2])

In [16]:
train_target_nodes

tensor([5, 4, 3, 5])

In [17]:
# Prepara os edges (arestas) de treino
train_edge_index = torch.stack([train_source_nodes, train_target_nodes], dim = 0)
train_edge_index = to_undirected(train_edge_index)
train_edge_attr = torch.tensor(train_ratings["rating"].values, dtype = torch.float32).view(-1, 1)

A função to_undirected tem como objetivo converter um grafo direcionado em um grafo não direcionado. Em outras palavras, ela garante que todas as arestas do grafo sejam bidirecionais, isto é, para cada aresta de A para B, existe também uma aresta de B para A.

Esta função pode ser útil em várias aplicações de aprendizado de máquina envolvendo grafos, onde os grafos não direcionados são necessários. Por exemplo, em problemas de clusterização de comunidades em redes sociais, análise de redes de coautoria, recomendação de itens e detecção de fraudes, pode ser necessário trabalhar com grafos não direcionados.

In [18]:
train_edge_index

tensor([[0, 1, 2, 3, 3, 4, 5, 5],
        [3, 4, 5, 0, 5, 1, 2, 3]])

In [19]:
train_edge_attr

tensor([[4.],
        [3.],
        [4.],
        [2.]])

In [20]:
# Repete o processo anterior para os dados de teste
test_source_nodes = torch.tensor(test_ratings["user_id"].values, dtype = torch.long)
test_target_nodes = torch.tensor(test_ratings["app_id"].values, dtype = torch.long)
test_edge_index = torch.stack([test_source_nodes, test_target_nodes], dim = 0)
test_edge_index = to_undirected(test_edge_index)
test_edge_attr = torch.tensor(test_ratings["rating"].values, dtype = torch.float32).view(-1, 1)

In [21]:
# Número de nodes
num_nodes = users.shape[0] + aplicativos.shape[0]

In [22]:
num_nodes

7

![DSA](imagens/grafo.png)

In [23]:
# Cria os atributos de aresta do conjunto de treinamento
train_data = Data(x = torch.eye(num_nodes, dtype = torch.float32), 
                  edge_index = train_edge_index, 
                  edge_attr = train_edge_attr, 
                  y = train_edge_attr)

In [24]:
train_data

Data(x=[7, 7], edge_index=[2, 8], edge_attr=[4, 1], y=[4, 1])

Essa representação acima é típica de um objeto Data do PyTorch Geometric, que armazena as informações de um grafo em um formato específico. Vamos entender cada componente dessa representação:

x=[7, 7]: Essa é a matriz de atributos dos nós. O primeiro valor (7) indica o número de nós no grafo, e o segundo valor (7) representa o número de atributos (características) para cada nó. Portanto, a matriz de atributos dos nós terá dimensões 7x7.

edge_index=[2, 8]: Essa é a matriz de índices de arestas, que indica as conexões entre os nós. A matriz tem duas linhas (valor 2), onde cada coluna representa uma aresta. O número 8 indica que existem 8 arestas no grafo.

edge_attr=[4, 1]: Essa é a matriz de atributos das arestas. O primeiro valor (4) indica o número de arestas no grafo, e o segundo valor (1) representa o número de atributos (características) para cada aresta. 

y=[4, 1]: Essa é a matriz de rótulos (labels) dos nós ou arestas, geralmente usada como a variável alvo (target) em problemas de aprendizado supervisionado. O primeiro valor (4) indica o número de rótulos, e o segundo valor (1) indica que cada rótulo é unidimensional (escalar). 

In [25]:
# Cria os atributos de aresta do conjunto de teste
test_data = Data(x = torch.eye(num_nodes, dtype = torch.float32), 
                 edge_index = test_edge_index, 
                 edge_attr = test_edge_attr, 
                 y = test_edge_attr)

In [26]:
test_data

Data(x=[7, 7], edge_index=[2, 2], edge_attr=[1, 1], y=[1, 1])

In [27]:
# Cria os dataloaders (o que é requerido pelo PyTorch)
train_data_loader = DataLoader([train_data], batch_size = 1)
test_data_loader = DataLoader([test_data], batch_size = 1)

## Construção do Modelo

https://arxiv.org/abs/1710.10903

In [28]:
# Classe do modelo
class RecommenderGAT(torch.nn.Module):
    
    # Método construtor
    def __init__(self, num_nodes, hidden_dim = 32, output_dim = 1, heads = 2):
        
        super(RecommenderGAT, self).__init__()
        
        # Primeira camada convolucional de atenção do grafo (GATConv)
        self.conv1 = GATConv(num_nodes, hidden_dim, heads = heads)
        
        # Segunda camada convolucional de atenção do grafo (GATConv)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads = heads)
        
        # Camada linear (fully connected) para produzir a saída final
        self.linear = torch.nn.Linear(hidden_dim * heads, output_dim)

    # Método forward
    def forward(self, data):
        
        # Extração dos atributos dos nós e da matriz de índices de arestas do objeto Data
        x, edge_index = data.x, data.edge_index

        # Aplicação da primeira camada GAT (self.conv1) seguida de uma função de ativação 
        x = self.conv1(x, edge_index)
        
        # Exponential Linear Unit (ELU)
        x = F.elu(x)
        
        # Aplicação de dropout para regularização (com probabilidade de 0,5)
        x = F.dropout(x, p = 0.5, training = self.training)

        x = self.conv2(x, edge_index)
        
        x = F.elu(x)

        # Agregação global para criar uma representação geral do grafo
        x = global_mean_pool(x, torch.zeros(1, dtype = torch.long))

        # Camada linear (self.linear) para gerar a saída final 
        x = self.linear(x)

        return x

In [29]:
# Cria instância do modelo
model = RecommenderGAT(num_nodes = num_nodes)

In [30]:
# Camadas do modelo
model.modules

<bound method Module.modules of RecommenderGAT(
  (conv1): GATConv(7, 32, heads=2)
  (conv2): GATConv(64, 32, heads=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)>

In [31]:
# Função de erro
loss_fn = torch.nn.MSELoss()

In [32]:
# Otimizador
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [33]:
# Número de épocas
num_epochs = 100

## Treinamento e Avaliação do Modelo

In [34]:
# Coloca o modelo em modo de treino
model.train()

RecommenderGAT(
  (conv1): GATConv(7, 32, heads=2)
  (conv2): GATConv(64, 32, heads=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [35]:
# Loop de treino
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_data_loader:
        optimizer.zero_grad()
        predictions = model(batch)
        loss = loss_fn(predictions, batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {total_loss / len(train_data_loader)}')

Epoch: 10, Loss: 2.941605806350708
Epoch: 20, Loss: 0.7767531871795654
Epoch: 30, Loss: 1.1417573690414429
Epoch: 40, Loss: 0.8151289224624634
Epoch: 50, Loss: 0.7017707824707031
Epoch: 60, Loss: 1.8063551187515259
Epoch: 70, Loss: 0.7335324287414551
Epoch: 80, Loss: 0.8187775015830994
Epoch: 90, Loss: 0.705970823764801
Epoch: 100, Loss: 0.7176841497421265


In [36]:
# Coloca o modelo em modo de avaliação
model.eval()

RecommenderGAT(
  (conv1): GATConv(7, 32, heads=2)
  (conv2): GATConv(64, 32, heads=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [37]:
# Faz previsões no conjunto de teste
with torch.no_grad():
    test_predictions = model(test_data).numpy()

In [38]:
# Calcula o RMSE comparando as previsões com as avaliações reais
rmse = np.sqrt(np.mean((test_predictions.flatten() - test_data.y.numpy().flatten()) ** 2))
print(f'RMSE no conjunto de teste: {rmse}')

RMSE no conjunto de teste: 2.102858304977417


## Deploy e Teste do Sistema de Recomendação

In [39]:
# Faz a previsão com base nos dados de teste
with torch.no_grad():
    previsao = model(test_data).numpy()

In [40]:
print(previsao)

[[2.8971417]]


In [41]:
# Cria um DataFrame com as previsões
df_previsoes = pd.DataFrame({
    "user_id": test_ratings["user_id"].values,
    "app_id": test_ratings["app_id"].values,
    "predicted_rating": np.round(previsao.flatten(), 0)
})

In [42]:
# Exibe a previsão
print(df_previsoes)

   user_id  app_id  predicted_rating
0        1       3               3.0


In [43]:
# Itera sobre as linhas do DataFrame
for index, row in df_previsoes.iterrows():
    if row["predicted_rating"] >= 3:
        print(f"Recomendamos o aplicativo {row['app_id']} para o usuário {row['user_id']}.")
    else:
        print(f"Não recomendamos o aplicativo {row['app_id']} para o usuário {row['user_id']}.")

Recomendamos o aplicativo 3.0 para o usuário 1.0.


# Fim